In [ ]:
# pip install torch==2.8.0

In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
import torch.nn.functional as F
from torch.autograd import grad

y = torch.tensor([1.0])
x1 = torch.tensor([1.1])
w1 = torch.tensor([2.2], requires_grad=True)
b = torch.tensor([0.0], requires_grad=True)
z = x1 * w1 + b
a = torch.sigmoid(z)
loss = F.binary_cross_entropy(a,y)

grad_L_w1 = grad(loss, w1, retain_graph=True)
grad_L_b = grad(loss, b, retain_graph=True)

print(grad_L_w1)
print(grad_L_b)

(tensor([-0.0898]),)
(tensor([-0.0817]),)


In [ ]:
class NeuralNetwork(torch.nn.Module):
  def __init__(self, num_inputs, num_outputs):
    super().__init__()

    self.layers = torch.nn.Sequential(
        # 1st hidden layer
        torch.nn.Linear(num_inputs, 30),
        torch.nn.ReLU(),

        # 2nd hidden layer
        torch.nn.Linear(30, 20),
        torch.nn.ReLU(),

        # output layer
        torch.nn.Linear(20, num_outputs)
    )

  def forward(self,x):
    logits = self.layers(x)
    return logits

In [ ]:
model = NeuralNetwork(50,3)
print(model)

NeuralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=50, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=3, bias=True)
  )
)


In [ ]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(num_params)

2213


In [ ]:
print(model.layers[0].weight.shape)
print(model.layers[0].bias)
print("웨이트", model.layers[0].weight)

torch.Size([30, 50])
Parameter containing:
tensor([ 0.0686, -0.1124, -0.0552,  0.0175,  0.1208,  0.0060, -0.0718, -0.0805,
         0.0298,  0.1265,  0.0572,  0.0155, -0.0779,  0.1048,  0.0364,  0.0634,
         0.0653, -0.0777, -0.0263,  0.0108,  0.0283,  0.0472, -0.0789,  0.0879,
         0.0992, -0.0632, -0.0304,  0.1109,  0.0696, -0.1079],
       requires_grad=True)
웨이트 Parameter containing:
tensor([[-0.0738, -0.0530, -0.0700,  ...,  0.0788,  0.0380,  0.0677],
        [-0.0625,  0.1219, -0.0963,  ...,  0.0637, -0.0427, -0.0504],
        [-0.1108, -0.0323,  0.0565,  ...,  0.0558,  0.0272, -0.0994],
        ...,
        [ 0.0247, -0.0619, -0.1332,  ...,  0.0879, -0.0381, -0.0023],
        [ 0.0057, -0.1156,  0.0679,  ..., -0.0575, -0.1168, -0.0015],
        [ 0.1379, -0.0271,  0.0413,  ...,  0.0484,  0.1193, -0.0857]],
       requires_grad=True)


In [ ]:
torch.manual_seed(123)
model = NeuralNetwork(50,3)
print(model.layers[0].weight)

Parameter containing:
tensor([[-0.0577,  0.0047, -0.0702,  ...,  0.0222,  0.1260,  0.0865],
        [ 0.0502,  0.0307,  0.0333,  ...,  0.0951,  0.1134, -0.0297],
        [ 0.1077, -0.1108,  0.0122,  ...,  0.0108, -0.1049, -0.1063],
        ...,
        [-0.0787,  0.1259,  0.0803,  ...,  0.1218,  0.1303, -0.1351],
        [ 0.1359,  0.0175, -0.0673,  ...,  0.0674,  0.0676,  0.1058],
        [ 0.0790,  0.1343, -0.0293,  ...,  0.0344, -0.0971, -0.0509]],
       requires_grad=True)


In [ ]:
torch.manual_seed(123)
x = torch.rand(1,50)
out = model(x)
print(out)

tensor([[-0.1262,  0.1080, -0.1792]], grad_fn=<AddmmBackward0>)


In [ ]:
with torch.no_grad():
  out = model(x)
  print(out)

tensor([[-0.1262,  0.1080, -0.1792]])


In [ ]:
with torch.no_grad():
  out = torch.softmax(model(x), dim = 1)
print(out)

tensor([[0.3113, 0.3934, 0.2952]])


In [ ]:
X_train = torch.tensor([[-1.2, 3.1],
                        [-0.9, 2.9],
                        [-0.5, 2.6],
                        [2.3, -1.1],
                        [2.7, -1.5]])
y_train = torch.tensor([0, 0, 0, 1, 1])
X_test = torch.tensor([[-0.8, 2.8],
                       [2.6,-1.6]])
y_test = torch.tensor([0,1])

In [ ]:
from torch.utils.data import Dataset

class ToyDataset(Dataset):
  def __init__(self,X,y):
    self.features = X
    self.labels = y

  def __getitem__(self,index):
    one_x = self.features[index]
    one_y = self.labels[index]
    return one_x, one_y

  def __len__(self):
      return self.labels.shape[0]

train_ds = ToyDataset(X_train, y_train)
test_ds = ToyDataset(X_test, y_test)

In [ ]:
from torch.utils.data import DataLoader

torch.manual_seed(123)

train_loader = DataLoader(
    dataset = train_ds,
    batch_size = 2,
    shuffle = True,
    num_workers = 0
)

test_loader = DataLoader(
dataset = test_ds,
batch_size = 2,
shuffle = False,
num_workers = 0
 )

In [ ]:
for idx, (x,y) in enumerate(train_loader):
  print(f"batch {idx+1} : ",x, y)

batch 1 :  tensor([[ 2.3000, -1.1000],
        [-0.9000,  2.9000]]) tensor([1, 0])
batch 2 :  tensor([[-1.2000,  3.1000],
        [-0.5000,  2.6000]]) tensor([0, 0])
batch 3 :  tensor([[ 2.7000, -1.5000]]) tensor([1])


In [ ]:
train_loader = DataLoader(
    dataset = train_ds,
    batch_size = 2,
    shuffle = True,
    num_workers = 0,
    drop_last = True
)

In [ ]:
for idx, (x,y) in enumerate(train_loader):
  print(f"batch {idx+1} : ",x, y)

batch 1 :  tensor([[-1.2000,  3.1000],
        [-0.5000,  2.6000]]) tensor([0, 0])
batch 2 :  tensor([[ 2.3000, -1.1000],
        [-0.9000,  2.9000]]) tensor([1, 0])


In [ ]:
# listing A.9 Neural network training in PyTorch

import torch.nn.functional as F

torch.manual_seed(123)
model = NeuralNetwork(num_inputs=2, num_outputs=2)
optimizer = torch.optim.SGD(model.parameters(), lr=0.5)

num_epochs = 3
for epoch in range(num_epochs):
  for batch_idx, (features,labels) in enumerate(train_loader):
    logits = model(features)

    loss = F.cross_entropy(logits, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    ### LOGGING
    print(f"Epoch: {epoch+1:03d}/{num_epochs:03d}"
    f" | Batch {batch_idx+1:03d}/{len(train_loader):03d}"
    f" | Train Loss: {loss:.2f}")

model.eval()


Epoch: 001/003 | Batch 001/002 | Train Loss: 0.75
Epoch: 001/003 | Batch 002/002 | Train Loss: 0.65
Epoch: 002/003 | Batch 001/002 | Train Loss: 0.44
Epoch: 002/003 | Batch 002/002 | Train Loss: 0.13
Epoch: 003/003 | Batch 001/002 | Train Loss: 0.03
Epoch: 003/003 | Batch 002/002 | Train Loss: 0.00


NeuralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=2, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=2, bias=True)
  )
)

In [ ]:
sum(p.numel() for p in model.parameters())

752

In [ ]:
model.eval()
with torch.no_grad():
  outputs = model(X_train)
print(outputs)

tensor([[ 2.8569, -4.1618],
        [ 2.5382, -3.7548],
        [ 2.0944, -3.1820],
        [-1.4814,  1.4816],
        [-1.7176,  1.7342]])


In [ ]:
torch.set_printoptions(sci_mode=False)
probas = torch.softmax(outputs, dim=1)
print(probas)

tensor([[    0.9991,     0.0009],
        [    0.9982,     0.0018],
        [    0.9949,     0.0051],
        [    0.0491,     0.9509],
        [    0.0307,     0.9693]])


In [ ]:
predictions = torch.argmax(probas, dim=1)
print(predictions)

tensor([0, 0, 0, 1, 1])


In [ ]:
predictions = torch.argmax(outputs, dim=1)
print(predictions)

tensor([0, 0, 0, 1, 1])


In [ ]:
predictions == y_train

tensor([True, True, True, True, True])

In [ ]:
torch.sum(predictions == y_train)

tensor(5)

In [ ]:
# A.10 function to compute the prediction accuracy

def compute_accuracy(model, dataloader):
  model = model.eval()
  correct = 0.0
  total_examples = 0

  for idx, (features, labels) in enumerate(dataloader):
    with torch.no_grad():
      logits = model(features)
    predictions = torch.argmax(logits, dim=1)
    compare = labels == predictions
    correct += torch.sum(compare)
    total_examples += len(compare)

  return (correct / total_examples).item()

In [ ]:
# A.8 Saving and loading models

torch.save(torch.nn.Module.state_dict(), "model.pth")
model = NeuralNetwork(2,2)
model.load_state_dict(torch.load("model.pth"))

TypeError: Module.state_dict() missing 1 required positional argument: 'self'

In [ ]:
# Listing A.11 A training loop on a GPU

torch.manual_seed(123)
model = NeuralNetwork(num_inputs=2, num_outputs=2)

device = torch.device("cuda")
model = model.to(device)

optimizer = torch.optim.SGD(model.parameters(), lr=0.5)

num_epochs = 3
for epoch in range(num_epochs):
  for batch_idx, (features,labels) in enumerate(train_loader):
    features = features.to(device)
    labels = labels.to(device)
    logits = model(features)

    loss = F.cross_entropy(logits, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    ### LOGGING
    print(f"Epoch: {epoch+1:03d}/{num_epochs:03d}"
    f" | Batch {batch_idx+1:03d}/{len(train_loader):03d}"
    f" | Train Loss: {loss:.2f}")

model.eval()

Epoch: 001/003 | Batch 001/002 | Train Loss: 0.75
Epoch: 001/003 | Batch 002/002 | Train Loss: 0.65
Epoch: 002/003 | Batch 001/002 | Train Loss: 0.44
Epoch: 002/003 | Batch 002/002 | Train Loss: 0.13
Epoch: 003/003 | Batch 001/002 | Train Loss: 0.03
Epoch: 003/003 | Batch 002/002 | Train Loss: 0.00


NeuralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=2, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=2, bias=True)
  )
)